<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/%EB%AF%B8%EC%85%9810.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 37.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import re
import string
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec, FastText

# 필요한 리소스 다운로드
nltk.download('wordnet')

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# 1.데이터 불러오기

In [ ]:
# 1. 데이터 불러오기
news_data = fetch_20newsgroups(subset='all',
# headers, footers, quotes를 제거해야 모델이 텍스트 내용에만 집중한다.
                               remove=('headers',
                                       'footers',
                                       'quotes')
                               )
texts = news_data.data     # 문서 내용-뉴스 본문(리스트)
labels = news_data.target  # 레이블-카테고리 정답(0~19 숫자)

In [ ]:
# news_data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
print('데이터 개수:', type(news_data.data), len(news_data.data))  # 뉴스 본문 리스트, 개수 출력
print('카테고리 개수:', type(news_data.target), len(news_data.target))
print(news_data.target_names)                    # 카테고리 리스트 출력
print(f"Unique labels: {set(labels)}")

데이터 개수: <class 'list'> 18846
카테고리 개수: <class 'numpy.ndarray'> 18846
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
Unique labels: {np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19)}


In [ ]:
# 데이터 분할
train_inputs, test_inputs, train_targets, test_targets = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
# stop_words = stopwords.words('english')

# 텍스트 데이터 전처리
def clean_text(text):
    # 소문자 변환
    text = text.lower()
    # 정규식: 알파벳과 공백만 남김
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # 불용어 제거
    stop_words = set(stopwords.words('english'))
    # 공백 기준 간단 토큰화
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# 데이터에 적용 (1)
train_inputs = [clean_text(text) for text in train_inputs]
test_inputs = [clean_text(text) for text in test_inputs]

# 단어 토큰화만 진행 (2)
# train_sentences = [word_tokenize(text) for text in train_inputs]
# test_sentences = [word_tokenize(text) for text in test_inputs]



# Word2vec

In [ ]:
# 단어 토큰화
train_sentences = [word_tokenize(text) for text in train_inputs]
test_sentences = [word_tokenize(text) for text in test_inputs]

In [ ]:
# Word2Vec 모델 학습
word2vec_model = Word2Vec(
    sentences=train_sentences,  # 학습 데이터 (토큰화된 문장 리스트)
    vector_size=128,            # 임베딩 차원 (128차원 벡터)
    window=5,                   # 주변 단어 고려 범위 (context window)
    min_count=1,                # 최소 등장 횟수 (1회 이상 등장한 단어만 학습)
    sg=1                        # 학습 알고리즘 (1=Skip-gram, 0=CBOW)
)
# 임베딩 행렬 초기화
word2vec_matrix = np.zeros((len(word2vec_model.wv) + 1, 128))
# 단어 → 인덱스 매핑 (word2idx)
word2idx_word2vec = {word: idx + 1 for idx, word in enumerate(word2vec_model.wv.index_to_key)}

for word, idx in word2idx_word2vec.items():
    word2vec_matrix[idx] = word2vec_model.wv[word]

In [ ]:
# Dataset 클래스 정의
class TextEmbeddingDataset(Dataset):
    def __init__(self, texts, labels, word2idx, max_len):
        self.texts = texts
        self.labels = labels
        self.word2idx = word2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = word_tokenize(self.texts[idx])
        # OOV 단어는 0
        encoded = [self.word2idx.get(word, 0) for word in tokens]  # OOV 단어는 0
        if len(encoded) < self.max_len:
            encoded += [0] * (self.max_len - len(encoded))# 패딩
        else:
            encoded = encoded[:self.max_len]# 잘라내기
        return torch.tensor(encoded, dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

In [ ]:
# DataLoader 준비
max_len = 280 # 분위수 기반으로 설정(95%:281.25)

# Word2Vec Dataset
train_dataset_word2vec = TextEmbeddingDataset(train_inputs, train_targets, word2idx_word2vec, max_len)
test_dataset_word2vec = TextEmbeddingDataset(test_inputs, test_targets, word2idx_word2vec, max_len)

# Word2Vec DataLoader
train_loader_word2vec = DataLoader(train_dataset_word2vec, batch_size=64, shuffle=True)
test_loader_word2vec = DataLoader(test_dataset_word2vec, batch_size=64, shuffle=False)

In [ ]:
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for texts, labels in loader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)  # 출력: (batch_size, 20)
        loss = criterion(outputs, labels)  # 레이블은 정수형 (0~19)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for texts, labels in loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            predictions = torch.argmax(outputs, dim=1)  # 가장 높은 확률의 인덱스
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return correct / total

In [ ]:
class EmbeddingLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, output_dim, num_layers=2, dropout=0.5):
        super(EmbeddingLSTM, self).__init__()
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(
            torch.tensor(embedding_matrix, dtype=torch.float).to(device),
            freeze=False # 학습 중 임베딩 업데이트 가능
            )
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim, output_dim) # 양방향 LSTM + hidden_dim*2

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        output = self.fc(hidden[-1])
        return output

In [ ]:
hidden_dim = 128
output_dim = len(set(labels))# 20개 카테고리

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_word2vec = EmbeddingLSTM(word2vec_matrix, hidden_dim, output_dim).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_word2vec.parameters(), lr=0.005)

for epoch in range(10):
    loss = train(model_word2vec, train_loader_word2vec, loss_fn, optimizer)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

accuracy = evaluate(model_word2vec, test_loader_word2vec)
print(f"Test Accuracy (Word2Vec with LSTM): {accuracy:.4f}")

cuda
Epoch 1, Loss: 2.0690
Epoch 2, Loss: 1.2437
Epoch 3, Loss: 0.7738
Epoch 4, Loss: 0.4933
Epoch 5, Loss: 0.3334
Epoch 6, Loss: 0.2397
Epoch 7, Loss: 0.1821
Epoch 8, Loss: 0.1709
Epoch 9, Loss: 0.1524
Epoch 10, Loss: 0.1287
Test Accuracy (Word2Vec with LSTM): 0.6430


# FastText

In [ ]:
# FastText 모델 학습
fasttext_model = FastText(sentences=train_sentences,
                          vector_size=128,
                          window=5,
                          min_count=1,
                          sg=1)
fasttext_matrix = np.zeros((len(fasttext_model.wv) + 1, 128))

word2idx_fasttext = {word: idx + 1 for idx, word in enumerate(fasttext_model.wv.index_to_key)}

for word, idx in word2idx_fasttext.items():
    fasttext_matrix[idx] = fasttext_model.wv[word]

In [ ]:
# FastText Dataset
train_dataset_fasttext = TextEmbeddingDataset(train_inputs, train_targets, word2idx_fasttext, max_len)
test_dataset_fasttext = TextEmbeddingDataset(test_inputs, test_targets, word2idx_fasttext, max_len)

# FastText DataLoader
train_loader_fasttext = DataLoader(train_dataset_fasttext, batch_size=64, shuffle=True)
test_loader_fasttext = DataLoader(test_dataset_fasttext, batch_size=64, shuffle=False)

In [ ]:
model_fasttext = EmbeddingLSTM(fasttext_matrix, hidden_dim, output_dim).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_fasttext.parameters(), lr=0.005)

for epoch in range(10):
    loss = train(model_fasttext, train_loader_fasttext, loss_fn, optimizer)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

accuracy = evaluate(model_fasttext, test_loader_fasttext)
print(f"Test Accuracy (FastText with LSTM): {accuracy:.4f}")

Epoch 1, Loss: 1.9575
Epoch 2, Loss: 1.1569
Epoch 3, Loss: 0.7266
Epoch 4, Loss: 0.4900
Epoch 5, Loss: 0.3428
Epoch 6, Loss: 0.2583
Epoch 7, Loss: 0.2018
Epoch 8, Loss: 0.1638
Epoch 9, Loss: 0.1387
Epoch 10, Loss: 0.1514
Test Accuracy (FastText with LSTM): 0.6645


# GloVe

In [ ]:
import os
import urllib.request

# GloVe 다운로드 및 임베딩 로드
GLOVE_URL = "https://nlp.stanford.edu/data/glove.6B.zip"
GLOVE_ZIP = "glove.6B.zip"
GLOVE_FILE = "glove.6B.200d.txt"

if not os.path.exists(GLOVE_FILE):
    print("Downloading GloVe embeddings...")
    urllib.request.urlretrieve(GLOVE_URL, GLOVE_ZIP)
    import zipfile
    with zipfile.ZipFile(GLOVE_ZIP, 'r') as zip_ref:
        zip_ref.extractall()
    print("GloVe embeddings downloaded and extracted.")
else:
    print("GloVe embeddings already available.")

GloVe embeddings downloaded and extracted.


In [ ]:
embedding_dim = 200  # GloVe 임베딩 차원 설정
glove_embeddings = {}

with open(GLOVE_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = coeffs

In [ ]:
len(glove_embeddings)

400000

In [ ]:
# GloVe word2idx 생성
word2idx_glove = {word: idx + 1 for idx, word in enumerate(glove_embeddings.keys())}
glove_matrix = np.zeros((len(word2idx_glove) + 1, embedding_dim))

for word, idx in word2idx_glove.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        glove_matrix[idx] = embedding_vector

In [ ]:
# GloVe Dataset
train_dataset_glove = TextEmbeddingDataset(train_inputs, train_targets, word2idx_glove, max_len)
test_dataset_glove = TextEmbeddingDataset(test_inputs, test_targets, word2idx_glove, max_len)

# GloVe DataLoader
train_loader_glove = DataLoader(train_dataset_glove, batch_size=32, shuffle=True)
test_loader_glove = DataLoader(test_dataset_glove, batch_size=32, shuffle=False)

In [ ]:
model_glove = EmbeddingLSTM(glove_matrix, hidden_dim, output_dim).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_glove.parameters(), lr=0.005)

for epoch in range(10):
    loss = train(model_glove, train_loader_glove, loss_fn, optimizer)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

accuracy = evaluate(model_glove, test_loader_glove)
print(f"Test Accuracy (GloVe with LSTM): {accuracy:.4f}")

Epoch 1, Loss: 1.7563
Epoch 2, Loss: 1.0265
Epoch 3, Loss: 0.6944
Epoch 4, Loss: 0.4894
Epoch 5, Loss: 0.3504
Epoch 6, Loss: 0.2661
Epoch 7, Loss: 0.2399
Epoch 8, Loss: 0.2442
Epoch 9, Loss: 0.2471
Epoch 10, Loss: 0.2013
Test Accuracy (GloVe with LSTM): 0.6618



## 📊 학습 곡선 해석

### 1. **Word2Vec**
- **Loss 감소**: 2.06 → 0.12까지 꾸준히 감소, 안정적으로 수렴
- **Test Accuracy**: **0.6430**
- → 학습은 잘 되었지만, 최종 정확도는 세 모델 중 가장 낮음

        Epoch 1, Loss: 2.0690
        Epoch 2, Loss: 1.2437
        Epoch 3, Loss: 0.7738
        Epoch 4, Loss: 0.4933
        Epoch 5, Loss: 0.3334
        Epoch 6, Loss: 0.2397
        Epoch 7, Loss: 0.1821
        Epoch 8, Loss: 0.1709
        Epoch 9, Loss: 0.1524
        Epoch 10, Loss: 0.1287
        Test Accuracy (Word2Vec with LSTM): 0.6430
---

### 2. **FastText**
- **Loss 감소**: 1.95 → 0.15까지 감소, Word2Vec과 비슷하게 안정적
- **Test Accuracy**: **0.6645**
- → 세 모델 중 **가장 높은 정확도**  
- 이유: FastText는 subword(부분 단어) 정보를 활용하기 때문에 희귀 단어, 오탈자, 형태 변화에 강함 → 일반화 성능이 조금 더 좋음

        Epoch 1, Loss: 1.9575
        Epoch 2, Loss: 1.1569
        Epoch 3, Loss: 0.7266
        Epoch 4, Loss: 0.4900
        Epoch 5, Loss: 0.3428
        Epoch 6, Loss: 0.2583
        Epoch 7, Loss: 0.2018
        Epoch 8, Loss: 0.1638
        Epoch 9, Loss: 0.1387
        Epoch 10, Loss: 0.1514
        Test Accuracy (FastText with LSTM): 0.6645
---

### 3. **GloVe**
- **Loss 감소**: 1.75 → 0.20까지 감소, 중간에 약간 흔들림(8~9 epoch에서 loss가 다시 올라감)
- **Test Accuracy**: **0.6618**
- → FastText보다 살짝 낮지만 Word2Vec보다는 높음  
- 이유: GloVe는 전역 동시출현 통계를 반영해 의미적 일관성이 좋음. 다만 학습 데이터와 도메인 차이가 있으면 성능이 흔들릴 수 있음

        Epoch 1, Loss: 1.7563
        Epoch 2, Loss: 1.0265
        Epoch 3, Loss: 0.6944
        Epoch 4, Loss: 0.4894
        Epoch 5, Loss: 0.3504
        Epoch 6, Loss: 0.2661
        Epoch 7, Loss: 0.2399
        Epoch 8, Loss: 0.2442
        Epoch 9, Loss: 0.2471
        Epoch 10, Loss: 0.2013
        Test Accuracy (GloVe with LSTM): 0.6618
---

## ✅ 종합 해석

| 임베딩 | 최종 정확도 | 특징 |
|--------|-------------|------|
| Word2Vec | 0.6430 | 단어 단위, 단순하고 빠르지만 OOV에 취약 |
| FastText | **0.6645 (최고)** | subword 기반, 희귀 단어/형태 변화에 강함 |
| GloVe | 0.6618 | 전역 통계 기반, 의미적 일관성 좋음 |

👉 **FastText가 가장 좋은 성능**을 보였고, GloVe가 그 뒤를 따르며, Word2Vec은 가장 낮은 정확도를 기록했습니다.  
즉, **데이터에 희귀 단어나 다양한 형태 변화가 많을수록 FastText가 유리**하다는 걸 보여주는 결과예요.


# 데이터 확인

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,937 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121713 files and dire

In [ ]:
# 샘플 문서 확인
print(f"Sample document: \n{texts[0]}")
# 정답 레이블 출력
print(news_data.target[0],news_data.target_names[news_data.target[0]])

Sample document: 


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


10 rec.sport.hockey


In [ ]:
# 전체 문서 수
print("전체 문서 수:", len(texts))

# 고유 문서 수 (중복 제거)
# unique_texts = set(texts)
print("고유 문서 수:", len(set(texts)))

# 중복 문서 수
print("중복 문서 수:", len(texts) - len(set(texts)))

# 예시: 중복된 문서 몇 개 출력해보기

duplicates = [doc for doc, count in Counter(texts).items() if count > 1]
print("중복된 문서 예시:", duplicates[:5])  # 앞에서 5개만 확인


전체 문서 수: 18846
고유 문서 수: 18287
중복 문서 수: 559
중복된 문서 예시: ['', 'please subscrive me.', 'Inguiry by address:er1@eridan.chuvashia.su\n', '\n\n\n\n\n', '\n\n\n']


In [ ]:

def check_duplicates(texts_, step_name):
    total = len(texts_)
    unique = len(set(texts_))
    duplicates = total - unique
    print(f"[{step_name}] 전체 문서 수: {total}, 고유 문서 수: {unique}, 중복 문서 수: {duplicates}")
    return duplicates

# 1. 원본 데이터
print("\n--- 전처리 단계별 중복 문서 변화 ---")
check_duplicates(texts, "원본")

# 2. 앞뒤 공백 제거
step1 = [t.strip() for t in texts]
check_duplicates(step1, "앞뒤 공백 제거")

# 3. 소문자 변환
step2 = [t.lower() for t in step1]
check_duplicates(step2, "소문자 변환")

# 4. 중간 공백 정규화
step3 = [re.sub(r'\s+', ' ', t) for t in step2]
check_duplicates(step3, "중간 공백 정규화")

# 5. 특수문자 제거 (예시)
step4 = [re.sub(r'[^a-zA-Z\s]', '', t) for t in step3]
check_duplicates(step4, "특수문자 제거")

# 6. 불용어 제거 (간단 예시)
stop_words = set(stopwords.words('english'))
step5 = [" ".join([w for w in t.split() if w not in stop_words]) for t in step4]
check_duplicates(step5, "불용어 제거")



--- 전처리 단계별 중복 문서 변화 ---
[원본] 전체 문서 수: 18846, 고유 문서 수: 18287, 중복 문서 수: 559
[앞뒤 공백 제거] 전체 문서 수: 18846, 고유 문서 수: 18263, 중복 문서 수: 583
[소문자 변환] 전체 문서 수: 18846, 고유 문서 수: 18260, 중복 문서 수: 586
[중간 공백 정규화] 전체 문서 수: 18846, 고유 문서 수: 18254, 중복 문서 수: 592
[특수문자 제거] 전체 문서 수: 18846, 고유 문서 수: 18232, 중복 문서 수: 614
[불용어 제거] 전체 문서 수: 18846, 고유 문서 수: 18205, 중복 문서 수: 641


641

# 2.데이터 전처리

In [ ]:
# 2-1. 토큰화 (시간이 좀 걸릴 수 있습니다)
def preprocess_text(text):
    # 1. 소문자 변환
    text = text.lower()

    # 2. 특수문자 제거 (알파벳과 공백만 남기기)
    # [^a-zA-Z\s]: 알파벳과 공백(스페이스, 엔터)이 '아닌' 것은 모두 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 3. 토큰화
    tokens = word_tokenize(text)

    # 4. 불용어 제거 (선택사항이지만 추천)
    tokens = [word for word in tokens if word not in stop_words]

    # 5. 길이가 너무 짧은 단어 제거 (예: 1글자)
    tokens = [word for word in tokens if len(word) > 1]

    return tokens
print("토큰화 진행 중...")
clean_texts = []
clean_labels = []

# zip을 써서 텍스트와 정답(label)을 같이 처리해야 짝이 맞습니다.
for text, label in zip(texts, labels):
    tokens = preprocess_text(text)

    # **중요**: 전처리 했더니 단어가 하나도 없는 경우(빈 리스트)는 데이터에서 제외
    # 전처리 결과가 비어있지 않은 경우에만 임시 저장
    if len(tokens) > 0:
        clean_texts.append(tokens)
        clean_labels.append(label)

print(f"원래 데이터 개수: {len(texts)}")
print(f"전처리 후 데이터 개수: {len(clean_texts)}")
# (전처리 과정에서 빈 텍스트가 제거되어 개수가 조금 줄어들 것입니다)

# 2. 중복 제거 (선택 사항이지만 추천)
# 리스트는 바로 set으로 못 바꾸므로, 튜플로 변환해서 중복 체크
unique_data = {}
for tokens, label in zip(clean_texts, clean_labels):
    # 토큰 리스트를 튜플로 변환 (Key로 쓰기 위해)
    tokens_tuple = tuple(tokens)

    # 딕셔너리에 저장하면 자동으로 중복된 Key(내용)는 덮어씌워짐
    # (내용이 같으면 라벨도 같다고 가정)
    unique_data[tokens_tuple] = label

# 3. 최종 데이터 복원
final_texts = [list(tokens) for tokens in unique_data.keys()]
final_labels = list(unique_data.values())

print(f"최종 학습 데이터 수(중복 제거 후): {len(final_texts)}")

# 변수명 업데이트 (이후 코드와 연결)
tokenized_texts = final_texts
labels = final_labels

# 데이터 확인
print("\n--- 최종 데이터 예시 ---")
print(f"Sample: {tokenized_texts[0]}")
print(f"Label: {labels[0]}")

# 2-2. 단어장(Vocab) 만들기
word_counts = Counter()
for text in tokenized_texts:
    word_counts.update(text)

# 빈도수가 낮은 단어는 제거하는 것이 성능에 좋습니다 (예: 5번 미만 등장 단어 무시)
vocab = {'<PAD>': 0, '<UNK>': 1} # PAD: 패딩용, UNK: 모르는 단어용
idx = 2
for word, count in word_counts.items():
    if count >= 5:
        vocab[word] = idx
        idx += 1

print(f"단어장 크기: {len(vocab)}")

# 2-3. 텍스트를 숫자로 변환 (Integer Encoding)
encoded_texts = []
for text in tokenized_texts:
    encoded = [vocab.get(word, vocab['<UNK>']) for word in text]
    encoded_texts.append(encoded)

# 2-4. 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    encoded_texts, labels, test_size=0.2, random_state=42, stratify=labels
)

토큰화 진행 중...
원래 데이터 개수: 18846
전처리 후 데이터 개수: 18299
최종 학습 데이터 수(중복 제거 후): 18200

--- 최종 데이터 예시 ---
Sample: ['sure', 'bashers', 'pens', 'fans', 'pretty', 'confused', 'lack', 'kind', 'posts', 'recent', 'pens', 'massacre', 'devils', 'actually', 'bit', 'puzzled', 'bit', 'relieved', 'however', 'going', 'put', 'end', 'nonpittsburghers', 'relief', 'bit', 'praise', 'pens', 'man', 'killing', 'devils', 'worse', 'thought', 'jagr', 'showed', 'much', 'better', 'regular', 'season', 'stats', 'also', 'lot', 'fo', 'fun', 'watch', 'playoffs', 'bowman', 'let', 'jagr', 'lot', 'fun', 'next', 'couple', 'games', 'since', 'pens', 'going', 'beat', 'pulp', 'jersey', 'anyway', 'disappointed', 'see', 'islanders', 'lose', 'final', 'regular', 'season', 'game', 'pens', 'rule']
Label: 10
단어장 크기: 25072


In [ ]:
from collections import Counter

# tokenized_texts는 이전 셀에서 전처리된 텍스트 리스트입니다.
# vocab 생성 시 사용했던 최소 빈도수 (min_count=5)를 고려합니다.

# 1. word_counts 다시 계산 (vocab 생성 로직과 동일하게 5회 이상 등장 단어만 고려)
all_words = []
for text_list in tokenized_texts:
    all_words.extend(text_list)

word_counts = Counter(all_words)

# vocab에 실제로 포함된 단어들만 필터링
filtered_word_counts = {word: count for word, count in word_counts.items() if count >= 5}

print("--- vocab 단어 빈도수 (많은 순) ---")
# 빈도수가 많은 순으로 정렬
sorted_by_most_frequent = sorted(filtered_word_counts.items(), key=lambda item: item[1], reverse=True)
for word, count in sorted_by_most_frequent[:10]:
    print(f"단어: '{word}', 빈도수: {count}")

print("\n--- vocab 단어 빈도수 (적은 순) ---")
# 빈도수가 적은 순으로 정렬
sorted_by_least_frequent = sorted(filtered_word_counts.items(), key=lambda item: item[1])
for word, count in sorted_by_least_frequent[:10]:
    print(f"단어: '{word}', 빈도수: {count}")


--- vocab 단어 빈도수 (많은 순) ---
단어: 'would', 빈도수: 10118
단어: 'one', 빈도수: 9933
단어: 'dont', 빈도수: 6413
단어: 'like', 빈도수: 6365
단어: 'people', 빈도수: 6276
단어: 'get', 빈도수: 5768
단어: 'know', 빈도수: 5726
단어: 'also', 빈도수: 5513
단어: 'think', 빈도수: 4975
단어: 'use', 빈도수: 4956

--- vocab 단어 빈도수 (적은 순) ---
단어: 'bashers', 빈도수: 5
단어: 'transfered', 빈도수: 5
단어: 'fuhrs', 빈도수: 5
단어: 'untrustworthy', 빈도수: 5
단어: 'laundry', 빈도수: 5
단어: 'augsburg', 빈도수: 5
단어: 'magick', 빈도수: 5
단어: 'apprehend', 빈도수: 5
단어: 'boasts', 빈도수: 5
단어: 'gyroscopes', 빈도수: 5


# 3.임베딩 적용

In [ ]:
# from gensim.models import Word2Vec

# 임베딩 차원 설정 (보통 100, 200, 300 등을 사용)
EMBEDDING_DIM = 100

# 1. Word2Vec 모델 학습 (현재 데이터셋으로)
w2v_model = Word2Vec(sentences=tokenized_texts,
                     vector_size=EMBEDDING_DIM,
                     window=5,
                     min_count=5,
                     workers=4
                     )
# 2. 임베딩 행렬(Weight Matrix) 생성
# 모델의 임베딩 레이어에 넣어줄 초기값입니다.
embedding_matrix = np.zeros((len(vocab), EMBEDDING_DIM))

for word, idx in vocab.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
    else:
        # Word2Vec에 없는 단어(UNK 등)는 랜덤 초기화
        embedding_matrix[idx] = np.random.normal(size=(EMBEDDING_DIM,))

# PyTorch Tensor로 변환
embedding_weights = torch.tensor(embedding_matrix, dtype=torch.float32)
print("임베딩 행렬 생성 완료")

임베딩 행렬 생성 완료


# 4.PyTorch Dataset & DataLoader 구현

In [ ]:
MAX_LEN = 200  # 문장의 최대 길이 설정 (너무 길면 자르고, 짧으면 0으로 채움)

class NewsDataset(Dataset):
    def __init__(self, texts, labels, max_len):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # 패딩 및 자르기 (Padding & Truncating)
        if len(text) > self.max_len:
            text = text[:self.max_len]
        else:
            text = text + [0] * (self.max_len - len(text)) # 0은 <PAD>의 인덱스

        return torch.tensor(text, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# 데이터 로더 생성
train_dataset = NewsDataset(X_train, y_train, MAX_LEN)
test_dataset = NewsDataset(X_test, y_test, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 5.LSTM 모델

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pretrained_weights):
        super(TextClassifier, self).__init__()

        # 1. 임베딩 레이어 (Pre-trained weights 로드)
        self.embedding = nn.Embedding.from_pretrained(pretrained_weights, freeze=False)
        # freeze=True면 학습 중에 임베딩이 변하지 않음, False면 미세조정됨

        # 2. LSTM 레이어
        # batch_first=True: 입력 데이터가 (Batch, Seq, Feature) 순서임
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # 3. 분류 레이어 (20개 카테고리)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: [batch_size, max_len]
        embedded = self.embedding(x)  # -> [batch_size, max_len, embedding_dim]

        # LSTM 통과
        # output: 모든 시점의 은닉 상태
        # (hidden, cell): 마지막 시점의 은닉 상태
        _, (hidden, cell) = self.lstm(embedded)

        # 마지막 시점의 hidden state를 사용하여 분류
        # hidden[-1] : [batch_size, hidden_dim]
        out = self.fc(hidden[-1])
        return out

# 모델 초기화
model = TextClassifier(
    vocab_size=len(vocab),
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=128,
    output_dim=20, # 뉴스 카테고리 수
    pretrained_weights=embedding_weights
)

# GPU 사용 가능 시 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 6.학습 및 평가 (Loop)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for batch in iterator:
        text, label = batch
        text, label = text.to(device), label.to(device)

        optimizer.zero_grad()
        predictions = model(text)
        loss = criterion(predictions, label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in iterator:
            text, label = batch
            text, label = text.to(device), label.to(device)

            predictions = model(text)
            loss = criterion(predictions, label)
            epoch_loss += loss.item()

            # 정확도 계산
            predicted_class = predictions.argmax(dim=1)
            correct += (predicted_class == label).sum().item()
            total += label.size(0)

    return epoch_loss / len(iterator), correct / total

# 학습 실행
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Epoch: 01 | Train Loss: 2.960 | Test Loss: 2.935 | Test Acc: 7.53%
Epoch: 02 | Train Loss: 2.693 | Test Loss: 2.416 | Test Acc: 21.51%
Epoch: 03 | Train Loss: 2.181 | Test Loss: 2.051 | Test Acc: 29.51%
Epoch: 04 | Train Loss: 1.826 | Test Loss: 1.781 | Test Acc: 39.20%
Epoch: 05 | Train Loss: 1.537 | Test Loss: 1.604 | Test Acc: 45.08%


In [ ]:
for epoch in range(20):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_loader, criterion)

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Epoch: 01 | Train Loss: 1.261 | Test Loss: 1.380 | Test Acc: 53.43%
Epoch: 02 | Train Loss: 0.989 | Test Loss: 1.300 | Test Acc: 58.54%
Epoch: 03 | Train Loss: 0.747 | Test Loss: 1.273 | Test Acc: 61.81%
Epoch: 04 | Train Loss: 0.579 | Test Loss: 1.266 | Test Acc: 63.52%
Epoch: 05 | Train Loss: 0.454 | Test Loss: 1.289 | Test Acc: 64.51%
Epoch: 06 | Train Loss: 0.351 | Test Loss: 1.318 | Test Acc: 65.82%
Epoch: 07 | Train Loss: 0.278 | Test Loss: 1.390 | Test Acc: 65.44%
Epoch: 08 | Train Loss: 0.224 | Test Loss: 1.467 | Test Acc: 65.25%
Epoch: 09 | Train Loss: 0.170 | Test Loss: 1.535 | Test Acc: 65.30%
Epoch: 10 | Train Loss: 0.146 | Test Loss: 1.542 | Test Acc: 66.81%
Epoch: 11 | Train Loss: 0.111 | Test Loss: 1.682 | Test Acc: 65.27%
Epoch: 12 | Train Loss: 0.088 | Test Loss: 1.706 | Test Acc: 65.22%
Epoch: 13 | Train Loss: 0.072 | Test Loss: 1.849 | Test Acc: 64.59%
Epoch: 14 | Train Loss: 0.075 | Test Loss: 1.807 | Test Acc: 65.66%
Epoch: 15 | Train Loss: 0.064 | Test Loss: 1.878

과적합-
65퍼센트 위로 못감


## 수정1

In [ ]:
class TextClassifier1(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pretrained_weights):
        super(TextClassifier1, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(pretrained_weights, freeze=False)

        # [수정 1] bidirectional=True 추가 (양방향 설정)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)

        # [수정 2] 입력 차원을 2배로 늘림 (hidden_dim * 2)
        # 왜냐하면 왼쪽에서 온 정보 + 오른쪽에서 온 정보가 합쳐지기 때문
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)

        # 출력값 받는 부분도 살짝 수정
        # hidden state 구조가 바뀌지만, 보통 output의 마지막 타임스텝을 쓰는 게 편함
        output, (hidden, cell) = self.lstm(embedded)

        # 양방향일 때는 output의 마지막 타임스텝을 가져오는 방식이 조금 다릅니다.
        # 가장 쉬운 방법: 마지막 hidden state 2개(정방향, 역방향)를 합쳐서 넣기
        hidden_cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)

        out = self.fc(hidden_cat)
        return out
# 모델 초기화
model1 = TextClassifier1(
    vocab_size=len(vocab),
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=128,
    output_dim=20, # 뉴스 카테고리 수
    pretrained_weights=embedding_weights
)

# GPU 사용 가능 시 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = model1.to(device)
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
criterion1 = nn.CrossEntropyLoss()
# 학습 실행
for epoch in range(20):
    train_loss = train(model1, train_loader, optimizer1, criterion1)
    test_loss, test_acc = evaluate(model1, test_loader, criterion1)

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Epoch: 01 | Train Loss: 2.202 | Test Loss: 1.944 | Test Acc: 35.08%
Epoch: 02 | Train Loss: 1.663 | Test Loss: 1.733 | Test Acc: 41.70%
Epoch: 03 | Train Loss: 1.396 | Test Loss: 1.380 | Test Acc: 55.22%
Epoch: 04 | Train Loss: 1.152 | Test Loss: 1.267 | Test Acc: 58.32%
Epoch: 05 | Train Loss: 0.916 | Test Loss: 1.223 | Test Acc: 60.49%
Epoch: 06 | Train Loss: 0.715 | Test Loss: 1.120 | Test Acc: 64.62%
Epoch: 07 | Train Loss: 0.530 | Test Loss: 1.107 | Test Acc: 66.40%
Epoch: 08 | Train Loss: 0.380 | Test Loss: 1.159 | Test Acc: 65.91%
Epoch: 09 | Train Loss: 0.272 | Test Loss: 1.176 | Test Acc: 68.82%
Epoch: 10 | Train Loss: 0.199 | Test Loss: 1.243 | Test Acc: 68.54%
Epoch: 11 | Train Loss: 0.135 | Test Loss: 1.336 | Test Acc: 67.80%
Epoch: 12 | Train Loss: 0.105 | Test Loss: 1.360 | Test Acc: 68.68%
Epoch: 13 | Train Loss: 0.067 | Test Loss: 1.489 | Test Acc: 66.92%
Epoch: 14 | Train Loss: 0.059 | Test Loss: 1.546 | Test Acc: 68.08%
Epoch: 15 | Train Loss: 0.043 | Test Loss: 1.586

## 수정2

In [ ]:
import torch
import numpy as np

class EarlyStopping:
    def __init__(self, patience=3, verbose=False, path='checkpoint.pt'):
        """
        patience: 성능이 안 좋아져도 참아주는 횟수 (보통 3~5)
        path: 최고 성능 모델을 저장할 파일 경로
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        # [수정] np.Inf -> np.inf (소문자로 변경)
        self.val_loss_min = np.inf
        self.path = path

    def __call__(self, val_loss, model):
        # 첫 실행이거나 성능이 더 좋아졌을 때
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss > self.best_loss:
            # 성능이 안 좋아짐 (Loss가 커짐) -> 참을성(Counter) 감소
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            # 성능이 다시 좋아짐 -> 카운터 초기화 및 저장
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''성능이 좋아졌을 때 모델 저장'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
# 1. 모델 정의 (오타 수정됨)
class TextClassifier2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pretrained_weights):
        # [수정] 클래스 이름에 맞춰서 super() 호출 (혹은 super().__init__() 사용)
        super(TextClassifier2, self).__init__()

        self.embedding = nn.Embedding.from_pretrained(pretrained_weights, freeze=False)

        # Dropout, 양방향, 2층 쌓기 적용됨
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2,
                            batch_first=True, bidirectional=True, dropout=0.5)

        self.dropout = nn.Dropout(0.5)

        # 양방향이므로 hidden_dim * 2
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)

        # 양방향의 마지막 hidden state 결합
        hidden_cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)

        hidden_cat = self.dropout(hidden_cat)
        out = self.fc(hidden_cat)
        return out

# 2. 모델 및 학습 설정
model2 = TextClassifier2(
    vocab_size=len(vocab),
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=128,    # 필요하면 256으로 늘려도 됨
    output_dim=20,
    pretrained_weights=embedding_weights
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2 = model2.to(device)

optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
criterion2 = nn.CrossEntropyLoss()

# [추가] 조기 종료 객체 생성 (patience=3: 성적이 3번 연속 안 오르면 멈춤)
# 위에서 만든 EarlyStopping 클래스가 정의되어 있어야 합니다.
early_stopping = EarlyStopping(patience=3, verbose=True, path='best_model.pt')

# 3. 학습 실행
print("학습 시작! (성능 향상 없으면 조기 종료됩니다)")
N_EPOCHS = 50  # 넉넉하게 잡아도 알아서 멈춤

for epoch in range(N_EPOCHS):
    train_loss = train(model2, train_loader, optimizer2, criterion2)
    test_loss, test_acc = evaluate(model2, test_loader, criterion2)

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

    # 조기 종료 체크
    early_stopping(test_loss, model2)
    if early_stopping.early_stop:
        print("조기 종료 발동! 과적합 직전에 멈췄습니다.")
        break

# 학습 종료 후, 가장 좋았던 모델 불러오기
model2.load_state_dict(torch.load('best_model.pt'))
print("최고 성능 모델 로드 완료.")

학습 시작! (성능 향상 없으면 조기 종료됩니다)
Epoch: 01 | Train Loss: 2.268 | Test Loss: 1.887 | Test Acc: 34.56%
Validation loss decreased (inf --> 1.886813).  Saving model ...
Epoch: 02 | Train Loss: 1.784 | Test Loss: 1.655 | Test Acc: 43.71%
Validation loss decreased (1.886813 --> 1.654714).  Saving model ...
Epoch: 03 | Train Loss: 1.548 | Test Loss: 1.479 | Test Acc: 49.34%
Validation loss decreased (1.654714 --> 1.479002).  Saving model ...
Epoch: 04 | Train Loss: 1.343 | Test Loss: 1.410 | Test Acc: 53.49%
Validation loss decreased (1.479002 --> 1.410051).  Saving model ...
Epoch: 05 | Train Loss: 1.124 | Test Loss: 1.280 | Test Acc: 57.45%
Validation loss decreased (1.410051 --> 1.279745).  Saving model ...
Epoch: 06 | Train Loss: 0.930 | Test Loss: 1.234 | Test Acc: 59.51%
Validation loss decreased (1.279745 --> 1.234161).  Saving model ...
Epoch: 07 | Train Loss: 0.751 | Test Loss: 1.188 | Test Acc: 62.66%
Validation loss decreased (1.234161 --> 1.187638).  Saving model ...
Epoch: 08 | Train 